In [1]:
from prep import data_prep

In [2]:
link=r'https://gist.githubusercontent.com/dbernaciak/1febd3fdc41db76dc9e7fed8ed157c2a/raw/cae64d5e18d452e968c7ec1ebf5b225f529f581a/Loan_Data.csv'
df=data_prep(url=link, target='default')
adjusted_loan_amounts=0.9*df.X_test['loan_amt_outstanding']

In [8]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score

model=LogisticRegressionCV()
model.fit(df.X_train,df.y_train)

logistic_probabilities=model.predict_proba(df.X_test)[:,1]

c:\Users\KhalelCorona\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\KhalelCorona\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sciki

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

params={'n_estimators':[1,5,25,125], 'max_depth':[1,2,3,4,5]}
model_2=RandomForestClassifier()
clf=GridSearchCV(estimator=model_2, param_grid=params, cv=5)
clf.fit(df.X_train, df.y_train)

forest_probabilities=clf.predict_proba(df.X_test)[:,1]

clf.best_params_

{'max_depth': 5, 'n_estimators': 125}

In [5]:
from sklearn.ensemble import GradientBoostingClassifier

params_2={'learning_rate':[0.05, 0.1, 0.15]}
model_3=GradientBoostingClassifier()
clf_2=GridSearchCV(estimator=model_3, param_grid=params_2, cv=5)
clf_2.fit(df.X_train, df.y_train)

gradient_probabilities=clf_2.predict_proba(df.X_test)[:,1]

clf_2.best_params_

{'learning_rate': 0.15}

In [9]:
from sklearn.metrics import accuracy_score
print(f' logistic accuracy score: {accuracy_score(model.predict(df.X_test), df.y_test)}')
print(f'forest accuracy score: {accuracy_score(clf.predict(df.X_test), df.y_test)}')
print(f'gradient accuracy score: {accuracy_score(clf_2.predict(df.X_test), df.y_test)}')

 logistic accuracy score: 0.9876
forest accuracy score: 0.9896
gradient accuracy score: 0.9944


In [11]:
from sklearn.ensemble import StackingClassifier
model_4=StackingClassifier(estimators=[('lr', model), ('gb', clf_2),
                                        ('rf',clf)])
model_4.fit(df.X_train, df.y_train)
stack_probabilities=model_4.predict_proba(df.X_test)

c:\Users\KhalelCorona\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\KhalelCorona\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sciki

In [12]:
print(f'stack accuracy score: {accuracy_score(model_4.predict(df.X_test), df.y_test)}')

stack accuracy score: 0.9956


In [14]:
expected_loss=stack_probabilities[:,1]*adjusted_loan_amounts
expected_loss

8324    2633.400896
3369    3041.043602
2043       6.861765
8426       9.463748
5072       7.434099
           ...     
351     1040.067896
6892       7.570525
5773    4619.984033
9302       6.701979
3619    3322.400389
Name: loan_amt_outstanding, Length: 2500, dtype: float64